In [ ]:
from alignment_calculation import loanbookMaker
from alignment_calculation import alignmentCalculator
from alignment_calculation import alignmentCalculatorConfig

%load_ext autoreload
%autoreload 2

In [ ]:
ac = alignmentCalculatorConfig()
ac.config(main_pacta_file={2022: ['../data/company_data/company_analytics.csv',
                                  '../data/company_data/company_indicators.csv'],
                          2023: ['../data/company_data/company_analytics.csv',
                                  '../data/company_data/company_indicators.csv'],},
          company_information_file='../data/company_data/company_information.csv')

In [ ]:
lm = loanbookMaker()

In [ ]:
year = 2023
month = 12
start_year = 2023
start_month = 1
jst_codes = []

In [ ]:
loans = lm.make_loanbook(start_year=year, start_month=month)

In [ ]:
ac = alignmentCalculator(loan_file=loans)

In [ ]:
res = ac.calculate_net_alignment(facet_col=['company_id','sector','technology'])

In [ ]:
ac.group_scores(res, ['portfolio_date','portfolio_code'])

In [ ]:
ac.group_scores(res, ['portfolio_date','sector'])